In [ ]:
import numpy as np

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm

In [ ]:
from IPython.display import display, clear_output

## data

In [ ]:
from astropy.table import Table

In [ ]:
data = Table.read('http://svo2.cab.inta-csic.es/vocats/alhambra/download/alhambra.csv.gz', format='ascii.csv')

In [ ]:
data_good = (data['zb_1'] <= 1.0) & (data['Stellar_Flag'] <= 0.5) & (data['Satur_Flag'] == 0)

In [ ]:
data_z, data_M = data[data_good]['zb_1'], data[data_good]['M_ABS_1']

In [ ]:
len(data_z)

In [ ]:
bins_z = np.linspace(0, 1, 10)
bins_M = np.linspace(-24, -16, 20)

In [ ]:
data_zM, _, _ = np.histogram2d(data_z, data_M, bins=[bins_z, bins_M])

In [ ]:
plt.imshow(data_zM.T, norm=LogNorm(), extent=(bins_z[0], bins_z[-1], bins_M[0], bins_M[-1]), aspect='auto')
plt.xlabel('redshift')
plt.ylabel('magnitude')
plt.clim(1, 1e4)
plt.colorbar()
plt.show()

In [ ]:
sigma_zM = np.sqrt(data_zM + 10)

## model

In [ ]:
%cat alhambra.yml

In [ ]:
from skypy.pipeline import Pipeline

In [ ]:
model = Pipeline.read('alhambra.yml')

In [ ]:
model.execute()

In [ ]:
model_z = np.concatenate([model['SF.z'], model['Q.z']])
model_M = np.concatenate([model['SF.M'], model['Q.M']])

In [ ]:
model_zM, _, _ = np.histogram2d(model_z, model_M, bins=[bins_z, bins_M])

In [ ]:
plt.imshow(data_zM.T, norm=LogNorm(), extent=(bins_z[0], bins_z[-1], bins_M[0], bins_M[-1]), aspect='auto')
plt.xlabel('redshift')
plt.ylabel('magnitude')
plt.clim(1, 1e4)
plt.colorbar()
plt.show()

plt.imshow(model_zM.T, norm=LogNorm(), extent=(bins_z[0], bins_z[-1], bins_M[0], bins_M[-1]), aspect='auto')
plt.xlabel('redshift')
plt.ylabel('magnitude')
plt.clim(1, 1e4)
plt.colorbar()
plt.show()

In [ ]:
parameters = model.parameters.copy()

## inference

In [ ]:
from scipy.optimize import minimize

In [ ]:
def metric(values):
    model.execute(zip(parameters.keys(), values))

    model_z = np.concatenate([model['SF.z'], model['Q.z']])
    model_M = np.concatenate([model['SF.M'], model['Q.M']])

    model_zM, _, _ = np.histogram2d(model_z, model_M, bins=[bins_z, bins_M])

    score = np.sum(np.square(model_zM - data_zM))
    
    clear_output(wait=True)
    display(f'last score: {score:12}')
    
    return score

In [ ]:
result = minimize(metric, list(parameters.values()), method='Nelder-Mead', options={'maxiter': 20})

In [ ]:
model.execute(zip(parameters.keys(), result.x))

In [ ]:
model_z = np.concatenate([model['SF.z'], model['Q.z']])
model_M = np.concatenate([model['SF.M'], model['Q.M']])

In [ ]:
model_zM, _, _ = np.histogram2d(model_z, model_M, bins=[bins_z, bins_M])

In [ ]:
plt.imshow(data_zM.T, norm=LogNorm(), extent=(bins_z[0], bins_z[-1], bins_M[0], bins_M[-1]), aspect='auto')
plt.xlabel('redshift')
plt.ylabel('magnitude')
plt.clim(1, 1e4)
plt.colorbar()
plt.show()

plt.imshow(model_zM.T, norm=LogNorm(), extent=(bins_z[0], bins_z[-1], bins_M[0], bins_M[-1]), aspect='auto')
plt.xlabel('redshift')
plt.ylabel('magnitude')
plt.clim(1, 1e4)
plt.colorbar()
plt.show()

In [ ]:
print(np.c_[list(parameters.values()), result.x])